First, import needed modules

In [204]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point
from geopy.geocoders import Nominatim
from shapely.geometry import LineString, Polygon
import os
from haversine import inverse_haversine, Direction, Unit

Initalize variables

In [205]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Load Dataset which has the endpoints of power line projects

In [206]:
powerlines = pd.read_csv(f'{path}/data/geodata/powerlines.csv', sep=';')
#powerlines = powerlines.set_index("# BBPlG 2013", drop = False)
powerlines.head()

,# BBPlG 2013,Start,End,New,Retro
0,3,Brunsbüttel,Großgartach,1,0
1,4,Wilster,Grafenrheinfeld,1,0
2,5,Bad Lauchstädt,Meitingen,1,0
3,17,Mecklar,Grafenrheinfeld,1,0


Get the geocoordinates of the 'endpoint' cities using OSM/Nominatim API

In [207]:
start_locations = geocode(powerlines['Start'], provider='nominatim', user_agent='plm', timeout=4)
start_locations.head()

,geometry,address
0,POINT (9.13954 53.89725),"Brunsbüttel, Dithmarschen, Schleswig-Holstein,..."
1,POINT (9.37338 53.92328),"Wilster, Kreis Steinburg, Schleswig-Holstein, ..."
2,POINT (11.86870 51.38813),"Bad Lauchstädt, Lindenstraße, Lindenstraße, Kl..."
3,POINT (9.76125 50.91746),"Mecklar, Ludwigsau, Landkreis Hersfeld-Rotenbu..."


In [208]:
end_locations = geocode(powerlines['End'], provider='nominatim', user_agent='plm', timeout=4)
end_locations.head()

,geometry,address
0,POINT (9.12587 49.14254),"Großgartach, Leingarten, Landkreis Heilbronn, ..."
1,POINT (10.19668 50.00553),"Grafenrheinfeld, Landkreis Schweinfurt, Bayern..."
2,POINT (10.85000 48.55000),"Meitingen, Landkreis Augsburg, Bayern, 86405, ..."
3,POINT (10.19668 50.00553),"Grafenrheinfeld, Landkreis Schweinfurt, Bayern..."


Merge into powerlines dataset

In [209]:
powerlines_geo_start = start_locations.join(powerlines, how = 'left')
powerlines_geo_start.rename(columns = {'geometry':'Start_geometry', 'address':'Start_address'}, inplace = True)
powerlines_geo = end_locations.join(powerlines_geo_start, how = 'left')
powerlines_geo.rename(columns = {'geometry':'End_geometry', 'address':'End_address'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['# BBPlG 2013', 'Start', 'Start_geometry', 'End', 'End_geometry', 'direct_line', 'New', 'Retro'])
powerlines_geo.head()

,# BBPlG 2013,Start,Start_geometry,End,End_geometry,direct_line,New,Retro
0,3,Brunsbüttel,POINT (9.13954 53.89725),Großgartach,POINT (9.12587 49.14254),NaN,1,0
1,4,Wilster,POINT (9.37338 53.92328),Grafenrheinfeld,POINT (10.19668 50.00553),NaN,1,0
2,5,Bad Lauchstädt,POINT (11.86870 51.38813),Meitingen,POINT (10.85000 48.55000),NaN,1,0
3,17,Mecklar,POINT (9.76125 50.91746),Grafenrheinfeld,POINT (10.19668 50.00553),NaN,1,0


Create direct line from start and end coordinates

In [210]:
for i in range (0, len(powerlines_geo)):
    start = powerlines_geo['Start_geometry'].loc[powerlines_geo.index[i]]
    end = powerlines_geo['End_geometry'].loc[powerlines_geo.index[i]]
    powerlines_geo['direct_line'].loc[powerlines_geo.index[i]] = LineString([start, end])
powerlines_geo.head()

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_25057/2775024114.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powerlines_geo['direct_line'].loc[powerlines_geo.index[i]] = LineString([start, end])
/opt/miniconda3/envs/python310/lib/python3.10/site-packages/pandas/core/internals/blocks.py:930: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return self.coerce_to_target_dtype(value).setitem(indexer, value)
/opt/miniconda3/envs/python310/lib/python3.10/site-packages/pandas/core/internals/managers.py:304: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  applied = getattr(b, f)(**kwargs)


,# BBPlG 2013,Start,Start_geometry,End,End_geometry,direct_line,New,Retro
0,3,Brunsbüttel,POINT (9.13954 53.89725),Großgartach,POINT (9.12587 49.14254),"LINESTRING (9.1395423 53.8972549, 9.1258701 49...",1,0
1,4,Wilster,POINT (9.37338 53.92328),Grafenrheinfeld,POINT (10.19668 50.00553),"LINESTRING (9.373378722135374 53.9232825, 10.1...",1,0
2,5,Bad Lauchstädt,POINT (11.86870 51.38813),Meitingen,POINT (10.85000 48.55000),"LINESTRING (11.8686979 51.3881334, 10.85 48.55)",1,0
3,17,Mecklar,POINT (9.76125 50.91746),Grafenrheinfeld,POINT (10.19668 50.00553),"LINESTRING (9.7612468 50.9174645, 10.1966808 5...",1,0


Export shapefile

In [211]:
powerlines_geo.rename(columns = {'direct_line':'geometry'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['#BBPlG2013', 'Start', 'End', 'geometry', 'New', 'Retro'])
powerlines_geo.to_file(f'{path}/data/geodata/powerlines.shp')

/opt/miniconda3/envs/python310/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
